# Importo i pacchetti da utilizzare

In [1]:
# beautifulsoup e requests servono per andare ad eseguire il web scraping della pagina
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

# Definizione funzioni utlizzate

In [2]:
# definisco una funzione per andare a riportare in un dataframe pandas il calendario con i risultati e i moduli delle squadre
# nella singola pagina
def partite(soup):
    squadre= soup.findAll("dt")
    
    tabellino_moduli= []
    for i in range(1,3):
        tabellino_moduli.append(squadre[i].get_text())
    
    ris= soup.findAll("title")
    risultato= ris[0].get_text()
    if ": tabellino, voti, pagelle e assist per il fantacalcio" in risultato:
        risultato= risultato.replace(": tabellino, voti, pagelle e assist per il fantacalcio","")
    if ": tabellino, voti, assist e pagelle per il Fantacalcio" in risultato:
        risultato= risultato.replace(": tabellino, voti, assist e pagelle per il Fantacalcio","")
    if ": tabellino, voti, assist e pagelle per il fantacalcio" in risultato:
        risultato= risultato.replace(": tabellino, voti, assist e pagelle per il fantacalcio","")
    
    tabellino_moduli.append(risultato)
    
    return tabellino_moduli
    

# estrapolo direttamente dalla pagina principale

In [3]:
url= "https://www.fantacalcio.it/serie-a"
page= requests.get(url).text
soup= BeautifulSoup(page, 'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js" lang="it-it">
 <head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# article: http://ogp.me/ns/article#">
  <!-- LayoutSitoWebInterna -->
  <title>
   Serie A - Fantacalcio.it
  </title>
  <meta content="Verona-Inter 2-2: tabellino, voti, assist e pagelle per il fantacalcio" name="description"/>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
  <meta content="app-id=636505602" name="apple-itunes-app"/>
  <meta content="app-id=it.Quadronica.Fantacalcio" name="google-play-app"/>
  <meta content="276105532435709" property="fb:app_id"/>
  <meta content="Fantacalcio.it - Solo il meglio del calcio" property="og:site_name"/>
  <meta content="Serie A - Fantacalcio.it" property="og:title"/>
  <meta content="Verona-Inter 2-2: tabellino, voti, assist e pagelle per il fantacalcio" property="og:descriptio

In [4]:
def makesoup(url):
    page= requests.get(url).text
    soup= BeautifulSoup(page, 'lxml')
    
    return soup

In [5]:
# creo una funzione che mi prenda tutte le partite che vi sono in una pagina
def partite_pag(url):
    page= requests.get(url).text
    soup_1= BeautifulSoup(page, 'lxml')
    
    tabellino_1= soup_1.find("h1", {"itemprop":"headline"})
    url_p= []
    for link in tabellino_1.findAll('a'):
        url_p.append(link.get('href'))
    url_partita= "https:" + str(url_p[0])
    soup= makesoup(url_partita)
    t_1= partite(soup)
    
    
    tabellino_2= soup_1.findAll("h3", {"itemprop":"headline"})
    url_partite= []
    for i in range(len(tabellino_2)):
        for link in tabellino_2[i]:
            url_partite.append(link.get('href'))
    
    url= []
    sp= []
    tabella= []
    for i in range(len(url_partite)):
        url.append("https:" + str(url_partite[i]))
        soup= makesoup(url[i])
        sp.append(soup)
        t_2= partite(sp[i])
        tabella.append(t_2)
        
    tabella.append(t_1)
    
    column= ['CASA', 'TRASFERTA', 'RISULTATO']
    tabella= pd.DataFrame(tabella, columns= column)

    
    return tabella
    



# eseguo su tutte le pagine

In [6]:
def tot_pag(url):
    newurl= []
    for p in range(2,13):
        newurl.append(url + "/" + str(p))
    
    pag1= partite_pag(url)
    pag2= partite_pag(newurl[0])
    pag3= partite_pag(newurl[1])
    pag4= partite_pag(newurl[2])
    pag5= partite_pag(newurl[3])
    pag6= partite_pag(newurl[4])
    pag7= partite_pag(newurl[5])
    pag8= partite_pag(newurl[6])
    pag9= partite_pag(newurl[7])
    pag10= partite_pag(newurl[8])
    pag11= partite_pag(newurl[9])
    pag12= partite_pag(newurl[10])
    
    tot= pd.concat([pag1, pag2, pag3, pag4, pag5, pag6, pag7, pag8, pag9,
                   pag10, pag11, pag12], ignore_index=True)
    
    return tot

In [7]:
tot_pag(url)

,CASA,TRASFERTA,RISULTATO
0,SPAL (3-5-1-1),Udinese (3-5-2),SPAL-Udinese 0-3
1,Atalanta (3-4-1-2),Sampdoria (4-5-1),Atalanta-Sampdoria 2-0
2,Roma (3-4-2-1),Parma (4-3-3),Roma-Parma 2-1
3,Bologna (4-2-3-1),Sassuolo (4-2-3-1),Bologna-Sassuolo 1-2
4,Torino (3-4-1-2),Brescia (4-3-1-2),Torino-Brescia 3-1
...,...,...,...
136,Roma (4-2-3-1),Torino (3-4-2-1),Roma-Torino ---
137,Genoa (3-5-2),Sassuolo (4-2-3-1),Genoa-Sassuolo 2-1
138,SPAL (3-5-2),Verona (3-4-2-1),Spal-Verona
139,Brescia (4-3-1-2),Lazio (3-5-2),Brescia-Lazio 1-2
